# 1. Importation des Bibliothèques

In [1]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import mlflow
import os


# 2. Chargement des Données

In [3]:
# Charger les données (en supposant que vous avez déjà les colonnes nettoyées)
data = pd.read_csv('../data/database_p7_rework.csv')

# Transformation des labels : 0 reste 0 et 4 devient 1
data['target_binary'] = data['target'].apply(lambda x: 0 if x == 0 else 1)

# Vérification des transformations
print(data['target_binary'].value_counts())

# Ensuite, définissez y comme suit :
y = data['target_binary']

target_binary
0    800000
1    798315
Name: count, dtype: int64


In [6]:
# Vérification des colonnes nécessaires
assert 'text_lemmatized' in data.columns, "La colonne 'text_lemmatized' est manquante"
assert 'text_stemmed' in data.columns, "La colonne 'text_stemmed' est manquante"


# 3. Préparation des Données
Tokenization avec BERT

In [ ]:
# Initialiser le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Fonction pour encoder un texte en utilisant BERT
def encode_with_bert(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=25)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# Appliquer BERT sur les descriptions lemmatisées et stemmées
bert_features_lemma = np.array([encode_with_bert(desc) for desc in data['text_lemmatized']])
bert_features_stem = np.array([encode_with_bert(desc) for desc in data['text_stemmed']])


D:\Master_Openclassroom\python\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`.

In [ ]:
# Séparation des données en ensembles d'entraînement et de test pour les données lemmatisées
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(
    bert_features_lemma, data['target_binary'], test_size=0.2, random_state=42)

# Séparation des données en ensembles d'entraînement et de test pour les données stemmées
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(
    bert_features_stem, data['target_binary'], test_size=0.2, random_state=42)


# Construction des modèles CNN et LSTM avec BERT

In [ ]:
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


# 4 Evaluation des modèles avec MLflow

In [ ]:
# Définir le dossier racine où vous voulez enregistrer les artefacts MLflow
base_dir = os.path.join("..", "mlruns")

# Fonction pour configurer MLflow pour chaque modèle
def configure_mlflow(model_name):
    mlflow_base_dir = os.path.join(base_dir, model_name)
    if not os.path.exists(mlflow_base_dir):
        os.makedirs(mlflow_base_dir)
    mlflow.set_tracking_uri(f"file://{mlflow_base_dir}")


In [ ]:
# Liste des modèles que vous allez entraîner
model_list = [
    ("cnn_model_lemma_bert", create_cnn_model((X_train_lemma.shape[1], X_train_lemma.shape[2]))),
    ("lstm_model_lemma_bert", create_lstm_model((X_train_lemma.shape[1], X_train_lemma.shape[2]))),
    ("cnn_model_stem_bert", create_cnn_model((X_train_stem.shape[1], X_train_stem.shape[2]))),
    ("lstm_model_stem_bert", create_lstm_model((X_train_stem.shape[1], X_train_stem.shape[2]))),
]

for model_name, model in model_list:
    configure_mlflow(model_name)
    
    with mlflow.start_run():
        if "lemma" in model_name:
            X_train, y_train, X_test, y_test = X_train_lemma, y_train_lemma, X_test_lemma, y_test_lemma
        else:
            X_train, y_train, X_test, y_test = X_train_stem, y_train_stem, X_test_stem, y_test_stem
        
        model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.2)
        mlflow.keras.log_model(model, model_name)
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)


In [ ]:
# Initialiser une liste pour stocker les résultats
results_bert = []

# Fonction pour ajouter les résultats d'un modèle à la liste
def log_results(model_name, accuracy, precision, recall, f1):
    results_bert.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

# Enregistrement des résultats pour chaque modèle
log_results("CNN + Lemmatization + BERT", accuracy_lemma_bert, precision_lemma_bert, recall_lemma_bert, f1_lemma_bert)
log_results("LSTM + Lemmatization + BERT", accuracy_lstm_lemma_bert, precision_lstm_lemma_bert, recall_lstm_lemma_bert, f1_lstm_lemma_bert)
log_results("CNN + Stemming + BERT", accuracy_stem_bert, precision_stem_bert, recall_stem_bert, f1_stem_bert)
log_results("LSTM + Stemming + BERT", accuracy_lstm_stem_bert, precision_lstm_stem_bert, recall_lstm_stem_bert, f1_lstm_stem_bert)

# Convertir les résultats en DataFrame
results_bert_df = pd.DataFrame(results_bert)

# Afficher le tableau des résultats
print(results_bert_df)

